In [5]:
from market_data_loader import MarketDataLoader
from order import Order
from order import risk_params
from oms import OrderManagementSystem
from order_book import LimitOrderBook
from position_tracker import PositionTracker
from strategies.trend_following   import run_backtest as tf_backtest
from strategies.mean_reversion    import run_backtest as mr_backtest
from strategies.arbitrage         import run_backtest as arb_backtest
import uuid

loader  = MarketDataLoader(interval="5m", period="1mo")
oms     = OrderManagementSystem()
tracker = PositionTracker()
book    = LimitOrderBook("AAPL")

rp = risk_params()

In [4]:
signals, trades, metrics = tf_backtest("AAPL", loader, risk_params, short_win=10, long_win=50)

[*********************100%***********************]  1 of 1 completed


In [11]:
for trade in trades:
    order_data = {
            "id": trade['order_id'],
            "symbol": trade["symbol"],
            "side": trade["side"],
            "quantity": trade["filled_qty"],  # rename key
            "type": "market",          # or whatever is appropriate
            "price": trade["price"],
            "timestamp": trade["timestamp"]
        }
    order = Order(**order_data)
    oms_ack = oms.new_order(order)
    exec_reports = book.add_order(order)
    for rpt in exec_reports:
        tracker.update(rpt)

In [13]:
blotter = tracker.get_blotter()  # DataFrame with columns: timestamp, symbol, side, qty, price, pnl
print(blotter.head())
blotter["cum_pnl"] = blotter["pnl"].cumsum()
blotter.set_index("timestamp")["cum_pnl"].plot(figsize=(10,5), title="Equity Curve")

Empty DataFrame
Columns: []
Index: []


KeyError: 'pnl'

In [ ]:
print(f"**Total Return:**  {metrics["total_return"]:.2%}")  
print(f"**Max Drawdown:**  {metrics["max_drawdown"]:.2%}")  
print(f"**Sharpe Ratio:**  {metrics["sharpe_ratio"]:.2f}")